In [76]:
import numpy as np

select_ingre = []

#재료 선택
def __init__(self, ingre):
    self.ingre = ingre

def print_info(self):
    print("select ingredient 다 고르셨으면 'r' 입력\n a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : ", self.ingre)

def select():
    ingre = input("select ingredient 다 고르셨으면 'r' 입력\n a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : ")
    if(ingre < 'a' or ingre > 'q'):
        print("없는 품목입니다 다시 고르세요\n")
    else :
        select_ingre.append(ingre)
    while(1):
        ingre = input("select ingredient 다 고르셨으면 'r' 입력\n a.사과 b.배 c.배추 d.무 e.양파 f.상추 g.오이 h.호박 i.쇠고기 j.돼지고기 k.닭고기 l.달걀(10개) m.달걀(30개) n.조기 o.명태 p.오징어 q.고등어 : ")
        if(ingre < 'a' or ingre > 'q' and ingre != 'r'):
            print("없는 품목입니다 다시 고르세요")
            continue
        elif(ingre == 'r'):
            break
        else:
            if ingre in select_ingre:
                print("중복입니다 다시 고르세요")
                continue
            else:
                select_ingre.append(ingre)
                continue
                
def run():
    select()
    
if __name__ == "__main__":
    run()

#선택한 재료 숫자 번호로 반환
for i in range (len(select_ingre)):
    select_ingre[i] = ord(select_ingre[i]) - ord('a')

#정렬
select_ingre.sort()
print(select_ingre)

#재료이름 순서대로 array에 저장
ingredient = np.array(['사과', '배', '배추', '무', '양파', '상추', '오이', '호박', '쇠고기',
                       '돼지고기', '닭고기', '달걀(10개)', '달걀(30개)', '조기', '명태', '오징어', '고등어'])
select_ingredient = []
for i in select_ingre:
    select_ingredient.append(ingredient[i])
print(select_ingredient)

#csv파일에서 시장이름정보 가져오기
all_market = np.genfromtxt('market_price_data.csv', delimiter = ',', usecols = 1, skip_header = 1, dtype = 'U')
market = np.zeros(49, dtype = 'U24')
for i in range(49):
    market[i] = all_market[(i + 1) * 17 - 17]
print (market)

#csv파일에서 재료 가격정보 가져오기, 배열에 저장, 행 : 시장, 열 : 재료
price = np.zeros((49, 17), dtype = 'i')
for i in range(49):
    price[i] = np.genfromtxt('market_price_data.csv', delimiter = ',', usecols = 3, skip_header = i*17+1, max_rows = 17)

#품목별 가장 작은 가격만 남겨두고 모두 0으로 변환, 원래 0인 값은 비교에서 제외
tmp = np.full((17), 100000)
for j in range(17):
    for i in range(49):
        if(price[i][j] <= tmp[j] and price[i][j] > 0):
            tmp[j] = price[i][j]
        else:
            price[i][j] = 0

for j in range(17):
    for i in range(49):
        if(price[i][j] > tmp[j]):
            price[i][j] = 0

print(tmp)
print(price)

#선택한 재료의 col만 따로 분리
select_price = price[:, select_ingre]
print(select_price)

#전치
select_price_T = select_price.T
print(select_price_T)

#가격정보가 0이 아닌 index를 찾는 과정
price_index_a = np.where(select_price > 0)
price_index_b = np.zeros((len(price_index_a[0]), 2), dtype = 'i')

a = 0
for i in range(len(select_price_T)):
    for j in range(49):
        if(select_price_T[i][j] != 0):
            price_index_b[a] = (i, j)
            a += 1

print(price_index_b)

#품목별 가장 싼 시장 출력
print("품목별 가장 싼 시장 목록")
for i in range(len(price_index_b)):
    print(select_ingredient[price_index_b[i][0]], "- ", market[price_index_b[i][1]])

#index의 상관관계를 이용하여 시장들의 부분 노드들을 구해보자
#select_market = []
select_tmp = []
select_tmp2 = []
select_tmp3 = []
mid_tmp = 1
for i in range(len(price_index_b) - 1):
    if(price_index_b[i][0] == price_index_b[i+1][0]):
        mid_tmp += 1
        if(i == (len(price_index_b) - 2)):
            select_tmp.append(mid_tmp)
    else:
        select_tmp.append(mid_tmp)
        mid_tmp = 1
        if(i == (len(price_index_b) - 2)):
            select_tmp.append(mid_tmp)

mid_tmp2 = 1
for i in range(len(select_tmp)):
    select_tmp2.append(mid_tmp2)
    mid_tmp2 *= select_tmp[len(select_tmp) - i - 1]
select_tmp3 = list(reversed(select_tmp2))

print(select_tmp)
print(select_tmp3)
print(mid_tmp)
print(mid_tmp2)

mid_tmp3 = 1
for i in range(len(select_tmp3)):
    mid_tmp3 *= select_tmp[i]
mid_tmp3 *= len(select_tmp)
print(mid_tmp3)

#시장들의 부분노드
select_market = np.zeros(mid_tmp3, dtype = 'i')
market_index = 0
for i in range(len(price_index_b)):
    tmp = price_index_b[i][0]
    if(market_index >= mid_tmp3):
        break
#    if(tmp == price_index_b[len(price_index_b) - 1][0]):
#        break
    if(select_tmp[tmp] == 1):
        for k in range(mid_tmp2):
            select_market[market_index] = price_index_b[i][1]
            market_index += 1
        if(market_index >= mid_tmp3):
            break
    else:
        for j in range(select_tmp3[tmp]):
            select_market[market_index] = price_index_b[i][1]
            market_index += 1
        if((market_index+1) / mid_tmp2 == 0)
        if(market_index >= mid_tmp3):
            break
#        select_market[market_index] = price_index_b[i][1]
#        market_index += 1
#        if(select_tmp[tmp] != 1):
#            select_tmp[tmp] -= 1
#            continue

#for i in range(int(mid_tmp2 / select_tmp[len(select_tmp) - 1])):
#    for j in range(len(price_index_b)):
#        if(price_index_b[j][0] == price_index_b[len(price_index_b) - 1][0]):
#            select_market[market_index] = price_index_b[j][1]
#            market_index += 1

print(select_market)
    
select_market_T = select_market.reshape(len(select_tmp), int(mid_tmp3 / len(select_tmp)))

market_result = select_market_T.T

print(market_result)


SyntaxError: invalid syntax (<ipython-input-76-50777862c822>, line 170)